In [62]:
import tables
from tables import *
import numpy as np
import glob
import tqdm 
import os


In [8]:
#retrieve params from our files
def read_files(inputfile):

    # read 1 column using field= parameter:
    with open_file(inputfile, "r", driver="H5FD_CORE") as h5f:
        img = h5f.root.dl1.event.telescope.images.LST_LSTCam.read(field="image")
        

    htable = np.fromfile("injunction_table_lst.pny", offset=8, dtype=np.uint16)
    matrix = np.zeros(55*55)

    mat = np.zeros((img.shape[0],55,55),dtype=np.float32)

    for i in range(img.shape[0]):
        matrix[htable] = img[i]
        res = matrix.reshape(55,55)
        mat[i] = res

    return mat

In [9]:
#return the singular values
def svd(res):
    # Applying SVD
    U, S, VT = np.linalg.svd(res,full_matrices=False,# It's not necessary to compute the full matrix of U or V
        compute_uv=True) # Deterministic SVD

    S = np.diag(S) #singular value matrix
    
    return S.diagonal()

In [10]:
def get_sv(train_fnames):
    nbevent = 0
    file_offset = []

    for file in tqdm.tqdm(train_fnames):
        hfile = tables.open_file(file, "r")
        nrows = hfile.root.dl1.event.telescope.parameters.LST_LSTCam.nrows
        file_offset.append((file,int(nbevent), int(nbevent+nrows)))
        nbevent = nbevent + nrows
        hfile.close()

    res = np.zeros((nbevent,10))
    
    for input in tqdm.tqdm(file_offset):
        mat = read_files(input[0])
        for i,img in enumerate(mat):
            sv = svd(img)
            res[input[1]+i] = sv[0:10]
    

    return res

In [11]:
def computeProjection(inputfile, datasetFName, Ncomponents):
    #retrieve principal components from projection file config
    with open_file(inputfile, "r", driver="H5FD_CORE") as h5f:
        pc = h5f.root.Projection.ACP.read(field="pc")
    #choose N principal components
    pc = pc[0:Ncomponents]
    #get singular values
    val = get_sv(datasetFName)
    # create matrix result
    res = np.zeros((val.shape[0],Ncomponents))
    #compute dot product
    for i,values in enumerate(val):
        for j,components in enumerate(pc):
            res[i][j] = np.dot(components,values)
    return res

In [12]:
train_fnames = glob.glob('Data/training/*.h5', recursive=True)
proj = computeProjection('TrainFiles/projection_svd.h5',train_fnames,3)

100%|██████████| 500/500 [27:57<00:00,  3.36s/it]


In [14]:
proj.shape

(1058742, 3)

In [41]:
#creating tables for datasets
class My_projection(IsDescription):
    firstCol = Float32Col()
    secondCol = Float32Col()
    thirdCol = Float32Col()

In [46]:
h5file = open_file("TrainDataProjection.h5",mode ="w", title="projection svd principal components")
#create a group
group = h5file.create_group("/",'Train','Train Info')
#create a table
table = h5file.create_table(group,"projection",My_projection,"dot product: projection")
#now let's fill the table with its values
projected = table.row #write data rows into the table 
for projection in proj:
    projected['AxeX'] = projection[0]
    projected['AxeY'] = projection[1]
    projected['AxeZ'] = projection[2]
    #insert a new gerbe record

    projected.append()

table.flush()

h5file.close()

In [47]:
test_fnames = glob.glob('Data/testing/*.h5', recursive=True)
proj_test = computeProjection('TestFiles/projection_svd.h5',test_fnames,3)

100%|██████████| 500/500 [27:00<00:00,  3.24s/it]


In [48]:
fileTest = open_file("TestDataProjection.h5",mode ="w", title="projection svd principal components")
#create a group
groupTest = fileTest.create_group("/",'Test','Test Info')
#create a table
tableTest = fileTest.create_table(groupTest,"projection",My_projection,"dot product: projection")
#now let's fill the table with its values
projectedTest = tableTest.row #write data rows into the table 
for projection in proj:
    projectedTest['AxeX'] = projection[0]
    projectedTest['AxeY'] = projection[1]
    projectedTest['AxeZ'] = projection[2]
    #insert a new projection record

    projectedTest.append()

tableTest.flush()

fileTest.close()

In [49]:
proj_test.shape

(1057648, 3)